In [11]:
import pandas as pd

df = pd.read_csv('./data/car-sim1/driving_log.csv')

df.head()

,/Users/dan/Desktop/car-sim1/IMG/center_2018_08_29_21_47_54_476.jpg,/Users/dan/Desktop/car-sim1/IMG/left_2018_08_29_21_47_54_476.jpg,/Users/dan/Desktop/car-sim1/IMG/right_2018_08_29_21_47_54_476.jpg,0,0.1,0.2,6.222208E-06
0,/Users/dan/Desktop/car-sim1/IMG/center_2018_08...,/Users/dan/Desktop/car-sim1/IMG/left_2018_08_2...,/Users/dan/Desktop/car-sim1/IMG/right_2018_08_...,0.0,0.000000,0,0.000012
1,/Users/dan/Desktop/car-sim1/IMG/center_2018_08...,/Users/dan/Desktop/car-sim1/IMG/left_2018_08_2...,/Users/dan/Desktop/car-sim1/IMG/right_2018_08_...,0.0,0.000000,0,0.000006
2,/Users/dan/Desktop/car-sim1/IMG/center_2018_08...,/Users/dan/Desktop/car-sim1/IMG/left_2018_08_2...,/Users/dan/Desktop/car-sim1/IMG/right_2018_08_...,0.0,0.000000,0,0.000004
3,/Users/dan/Desktop/car-sim1/IMG/center_2018_08...,/Users/dan/Desktop/car-sim1/IMG/left_2018_08_2...,/Users/dan/Desktop/car-sim1/IMG/right_2018_08_...,0.0,0.000000,0,0.000010
4,/Users/dan/Desktop/car-sim1/IMG/center_2018_08...,/Users/dan/Desktop/car-sim1/IMG/left_2018_08_2...,/Users/dan/Desktop/car-sim1/IMG/right_2018_08_...,0.0,0.099603,0,0.068894


In [14]:
import csv
import cv2
import numpy as np

lines = []
with open('./data/car-sim1/driving_log.csv') as csvFile:
    reader = csv.reader(csvFile)
    next(reader, None)
    for line in reader:
        lines.append(line)
        
        
images = []
measurements = []
for line in lines:
    source_path = line[0]
    filename = source_path.split('/')[-1]
    current_path = './data/car-sim1/IMG/' + filename
    image = cv2.imread(current_path)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)
    
X_train = np.array(images)
Y_train = np.array(measurements)

print('X_train:', X_train.shape)
print('Y_train:', Y_train.shape)

X_train: (2286, 160, 320, 3)
Y_train: (2286,)


## First Model

In [16]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.optimizers import Adam

model = Sequential()
#model.add(Flatten(input_shape=(160, 320, 3)))
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Flatten())
model.add(Dense(1))

model.compile(loss='mse', optimizer=Adam(lr=0.0001))
model.fit(X_train, Y_train, validation_split=0.2, shuffle=True, epochs=5)

model.save('model.h5')

Train on 1828 samples, validate on 458 samples
Epoch 1/5
1828/1828 [==============================] - 1s 805us/step - loss: 0.1135 - val_loss: 0.1430
Epoch 2/5
1828/1828 [==============================] - 1s 667us/step - loss: 0.0570 - val_loss: 0.1118
Epoch 3/5
1828/1828 [==============================] - 1s 656us/step - loss: 0.0338 - val_loss: 0.0960
Epoch 4/5
1828/1828 [==============================] - 1s 660us/step - loss: 0.0235 - val_loss: 0.1027
Epoch 5/5
1828/1828 [==============================] - 1s 655us/step - loss: 0.0182 - val_loss: 0.0976


In [21]:
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Conv2D(6, (5,5), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(6, (5,5), activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

model.compile(loss='mse', optimizer=Adam(lr=0.001))
model.fit(X_train, Y_train, validation_split=0.2, shuffle=True, epochs=5)

model.save('model.h5')

Train on 1828 samples, validate on 458 samples
Epoch 1/5
1828/1828 [==============================] - 3s 2ms/step - loss: 16.9033 - val_loss: 0.1001
Epoch 2/5
1828/1828 [==============================] - 2s 882us/step - loss: 0.0470 - val_loss: 0.0598
Epoch 3/5
1828/1828 [==============================] - 2s 901us/step - loss: 0.0366 - val_loss: 0.0579
Epoch 4/5
1828/1828 [==============================] - 2s 893us/step - loss: 0.0352 - val_loss: 0.0559
Epoch 5/5
1828/1828 [==============================] - 2s 900us/step - loss: 0.0341 - val_loss: 0.0542


In [10]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1888059787822219105
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 302448640
locality {
  bus_id: 1
}
incarnation: 3621040110653449976
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:09:00.0, compute capability: 6.1"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 338100224
locality {
  bus_id: 1
}
incarnation: 4712016052853008706
physical_device_desc: "device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:0a:00.0, compute capability: 6.1"
, name: "/device:GPU:2"
device_type: "GPU"
memory_limit: 338100224
locality {
  bus_id: 1
}
incarnation: 4475804696287490450
physical_device_desc: "device: 2, name: GeForce GTX 1080 Ti, pci bus id: 0000:42:00.0, compute capability: 6.1"
, name: "/device:GPU:3"
device_type: "GPU"
memory_limit: 338100224
locality {
  bus_id: 1
}
incarnation: 10865371839291708873
physical_device_desc: "device: 3, name: GeForce GTX